# User Defined Equations

Examples:

- find value of evaporation temperature to achieve specific COP
- a custom superheat specification
- set the minimal pinch across a number of heat exchangers

## General info

- Formulate an equation $0 = ...$ in a custom function and return residual
- Specify which variables the equation depends on in a second function
- Pass all other relevant information to an instance of UserDefinedEquation
  - connections available through ude.conns
  - components available through ude.comps
  - function to calculate the partial derivatives explicitly instead of doing
    it via dependents
  - other parameters (dictionary) available through ude.params
- For more, readthedocs: https://tespy.readthedocs.io/en/main/modules/ude.html

## Specification of COP

- heat pump
- fixed efficiency of compressor
- fixed superheat and subcooling
- fixed evaporation and condensation temperature level
- target amount of heat production by condenser

In [1]:
from tespy.components import SimpleHeatExchanger, CycleCloser, Valve, Compressor
from tespy.connections import Connection
from tespy.networks import Network

In [2]:
nw = Network()
nw.units.set_defaults(
    temperature="°C",
    pressure="bar",
    power="kW",
    heat="kW"
)

In [3]:
evaporator = SimpleHeatExchanger("evaporator")
superheater = SimpleHeatExchanger("superheater")
compressor = Compressor("compressor")
condenser = SimpleHeatExchanger("condenser")
valve = Valve("valve")
cc = CycleCloser("cc")

c1 = Connection(evaporator, "out1", superheater, "in1", label="c1")
c1b = Connection(superheater, "out1", compressor, "in1", label="c1b")
c2 = Connection(compressor, "out1", condenser, "in1", label="c2")
c3 = Connection(condenser, "out1", valve, "in1", label="c3")
c4 = Connection(valve, "out1", cc, "in1", label="c4")
c5 = Connection(cc, "out1", evaporator, "in1", label="c5")

nw.add_conns(c1, c1b, c2, c3, c4, c5)

In [4]:
c1.set_attr(fluid={"R600": 1}, T_dew=10, td_dew=2)
c1b.set_attr(td_dew=12)
c3.set_attr(T_bubble=60, td_bubble=5)

evaporator.set_attr(dp=0)
superheater.set_attr(dp=1)
condenser.set_attr(dp=0, Q=-100)
compressor.set_attr(eta_s=0.8)

nw.solve("design")


 iter  | residual   | progress   | massflow   | pressure   | enthalpy   | fluid      | component  
-------+------------+------------+------------+------------+------------+------------+------------
 1     | 5.93e+05   | 2 %        | 1.83e+00   | 0.00e+00   | 2.30e+04   | 0.00e+00   | 0.00e+00   
 2     | 4.21e+04   | 15 %       | 1.13e-01   | 0.00e+00   | 9.09e-11   | 0.00e+00   | 0.00e+00   
 3     | 4.71e-10   | 100 %      | 1.32e-15   | 0.00e+00   | 9.09e-11   | 0.00e+00   | 0.00e+00   
 4     | 7.84e-11   | 100 %      | 1.26e-17   | 0.00e+00   | 9.09e-11   | 0.00e+00   | 0.00e+00   
Total iterations: 4, Calculation time: 0.00 s, Iterations per second: 1501.59


In [5]:
abs(condenser.Q.val) / compressor.P.val

2.8616514142579312

In [6]:
from tespy.tools import UserDefinedEquation


c1.set_attr(T_dew=None)

def cop_ude(ude):
    c1, c2, c3 = ude.conns
    target_cop = ude.params["target_cop"]
    return target_cop * (c2.h.val_SI - c1.h.val_SI) - (c2.h.val_SI - c3.h.val_SI)

def cop_ude_dependents(ude):
    return [c.h for c in ude.conns]

cop = 7

myude = UserDefinedEquation(
    "cop ude",
    cop_ude,
    cop_ude_dependents,
    conns=[c1, c2, c3],
    params={"target_cop": cop}
)

nw.add_ude(myude)
nw.solve("design")


 iter  | residual   | progress   | massflow   | pressure   | enthalpy   | fluid      | component  
-------+------------+------------+------------+------------+------------+------------+------------
 1     | 3.64e+05   | 4 %        | 2.15e-02   | 9.51e+04   | 7.61e+04   | 0.00e+00   | 0.00e+00   
 2     | 3.67e+04   | 15 %       | 9.00e-03   | 8.21e+04   | 1.70e+04   | 0.00e+00   | 0.00e+00   
 3     | 6.27e+03   | 24 %       | 1.20e-03   | 2.32e+04   | 2.55e+03   | 0.00e+00   | 0.00e+00   
 4     | 2.55e+02   | 39 %       | 4.07e-05   | 1.12e+03   | 7.96e+01   | 0.00e+00   | 0.00e+00   
 5     | 5.21e-01   | 69 %       | 8.12e-08   | 2.38e+00   | 1.48e-01   | 0.00e+00   | 0.00e+00   
 6     | 2.73e-06   | 100 %      | 8.20e-13   | 1.31e-05   | 1.53e-06   | 0.00e+00   | 0.00e+00   
 7     | 7.97e-10   | 100 %      | 2.80e-17   | 2.01e-09   | 4.73e-10   | 0.00e+00   | 0.00e+00   
Total iterations: 7, Calculation time: 0.02 s, Iterations per second: 421.45


In [7]:
abs(condenser.Q.val) / compressor.P.val

7.276456922425378

In [8]:
myude.params["target_cop"] = 4
nw.solve("design")
c1.T.val


 iter  | residual   | progress   | massflow   | pressure   | enthalpy   | fluid      | component  
-------+------------+------------+------------+------------+------------+------------+------------
 1     | 1.53e+05   | 9 %        | 1.87e-03   | 2.54e+05   | 6.16e+04   | 0.00e+00   | 0.00e+00   
 2     | 9.84e+03   | 22 %       | 3.98e-03   | 1.83e+03   | 6.94e+03   | 0.00e+00   | 0.00e+00   
 3     | 2.97e+01   | 50 %       | 6.29e-05   | 5.61e+00   | 1.13e+01   | 0.00e+00   | 0.00e+00   
 4     | 4.81e-04   | 100 %      | 1.29e-09   | 2.43e-04   | 1.17e-04   | 0.00e+00   | 0.00e+00   
 5     | 9.73e-10   | 100 %      | 5.51e-16   | 2.09e-09   | 1.81e-09   | 0.00e+00   | 0.00e+00   
Total iterations: 5, Calculation time: 0.01 s, Iterations per second: 450.79


17.11179350291286